In [59]:
class Vector:
    
    # Would like to say vector[1] instead of vector.elements[1]
    
    def __init__(self, *args):
        self.elements = []
        for arg in args:
            self.elements.append(arg)
            
    def size(self):
        return len(self.elements)
            
    def __str__(self):
        to_return = '['
        for i in range(self.size()):
            if i == self.size() - 1:
                to_return += f"{self.elements[i]}"
                continue
            to_return += f"{self.elements[i]} "
        to_return += ']'
        return to_return
    
    def __add__(self, arg):
        if self.size() != arg.size():
            print("Vectors must be same size to add")
            return
        sum = Vector(self.elements[0]+arg.elements[0])
        for i in range(1,self.size()):
           sum.elements.append(self.elements[i]+arg.elements[i])
        return sum
    
    def __mul__(self, arg): # Scalar Multiplication
        for i in range(self.size()):
            self.elements[i] *= arg
        return self
    
    def __sub__(self, arg):
        neg_arg = arg * (-1)
        return self + neg_arg
                
    def dot(self, arg): # Standard Dot Product
        if self.size() != arg.size():
            print("Vectors must be same size to multiply")
            return
        to_return = 0
        for i in range(self.size()):
            to_return += self.elements[i] * arg.elements[i]
        return to_return
    
    def norm(self):
        return self.dot(self)**(0.5)
    
    def normalize(self):
        return self * (1 / self.norm())
    
         

a = Vector(1,2,5)
b = Vector(1,1,1)
c = Vector(3,3,3)


[1 2 5]
[1 1 1]
===


AttributeError: 'Vector' object has no attribute 'degree'

In [118]:
class Polynomial(Vector):
    def __init__(self, *args):
        super().__init__(*args)
        
    # Should be able to solve for roots - reference abstract algebra
    
    def deg(self):
        return self.size() - 1 # This is easy to trick...
    
    def __str__(self):
        to_return = ''
        for i in range(self.deg() + 1):
            to_return += f"{self.elements[i]}x^{i}"
            if i != self.deg():
                to_return += ' + '
        return to_return
    
    def __mul__(self, arg):
        
        prod_deg = self.deg() + arg.deg()
        
        self_tmp = Polynomial(self.elements[0])
        for i in range(1, self.deg() + 1):
            self_tmp.elements.append(self.elements[i])
        for j in range(self.deg() + 1, prod_deg + 1):
            self_tmp.elements.append(0)
        
        arg_tmp = Polynomial(arg.elements[0])
        for i in range(1, arg.deg() + 1):
            arg_tmp.elements.append(arg.elements[i])
        for j in range(arg.deg() + 1, prod_deg + 1):
            arg_tmp.elements.append(0)
        
        product = Polynomial(self.elements[0]*arg.elements[0])
        current_coeff = 0
        for n in range(1, prod_deg + 1):
            current_coeff = 0
            for k in range(0, n + 1):
                current_coeff += self_tmp.elements[k]*arg_tmp.elements[n-k]
            product.elements.append(current_coeff)
        return product
        
    def __eq__(self, arg):
        
        # If different degrees, not equal, but if different sizes, not necessarily not equal
        # Can this detect different forms? Are there any different forms?
        pass
    
    def eval_at(self, arg: int):
        to_return = 0
        for i in range(self.deg() + 1):
            to_return += self.elements[i]*arg**i
        return to_return
        


x = Polynomial(0, 1, 4)
y = Polynomial(4, 2, 6, 6)

# Vector addition works but I've defined it to only allow addition of same size vectors 
# which doesnt make as much sense with polynomials - should be able to add polynomials of different degree


In [83]:
# Need to make sure this doesnt change determinant function
# Probably gonna want to redefine determinant function as a class function

# In definition, is there a way to ensure its not more than 2D? 

class Matrix:
    def __init__(self, elements: list): # Should I use *args?
        for i in range(len(elements)):
            if len(elements[0]) != len(elements[i]): # requires one dimensional matrices be defined as [[1]] instead of [1]
                print("Invalid. Matrix must be rectangluar")
        self.elements = elements    
        self.rows = len(elements)
        self.cols = len(elements[0])
        
    
    def __str__(self) -> str:
        to_return = "[ "
        for i in range(self.rows):
            for j in range(self.cols):
                to_return +=  f"{self.elements[i][j]} "
            if i != self.rows - 1:
                to_return += "\n  "
        to_return += "]"
        return to_return
    
    def __add__(self, arg): # Check commutativity
        
        if self.rows != arg.rows or self.cols != arg.cols:
            print("Matrices must be the same size.")
        
        rows = self.rows
        cols = self.cols
        sum_elements = [[0 for _ in range(cols)] for _ in range(rows)]
        
        for i in range(rows):
            for j in range(cols):
                sum_elements[i][j] = self.elements[i][j] + arg.elements[i][j]
                
        sum = Matrix(sum_elements)
        return sum
    
    def scale(self, arg: int): # Scalar Multiplication Helper Function for __mul__ case of type(arg)==int
        rows = self.rows
        cols = self.cols
        scaled_elements = [[0 for _ in range(cols)] for _ in range(rows)]
        for i in range(rows):
            for j in range(cols):
                scaled_elements[i][j] = arg * self.elements[i][j]
        scaled_matrix = Matrix(scaled_elements)
        return scaled_matrix
    
    def __mul__(self, arg):
        if (type(arg)==int): # Scalar Multiplication 
            return self.scale(arg)
        
        # Matrix Multiplication, Returns Self * Arg, in this order
        if self.cols != arg.rows:
            print("Matrices of these sizes cannot be multiplied in this order.")
        
        rows = self.rows
        cols = arg.cols
        product_elements = [[0 for _ in range(cols)] for _ in range(rows)] # 2D list of all zeros
        tmp = 0
        
        for i in range(rows):
            for j in range(cols):
                for k in range(self.cols):
                    tmp += self.elements[i][k] * arg.elements[k][j]
                product_elements[i][j] = tmp
                tmp = 0
        
        return Matrix(product_elements)
    
    def __pow__(self, arg):
        rows = self.rows
        cols = self.cols
        tmp_matrix = [[0 for _ in range(cols)] for _ in range(rows)]
        for i in range(rows):
            for j in range(cols):
                if i == j:
                    tmp_matrix[i][j] = 1
        Identity = Matrix(tmp_matrix)
        
        for i in range(arg):
            Identity *= self
        return Identity # This is self^arg
            
    
    def rref(self): # This can be used for a linear system solver
        pass
    
    
A = Matrix([[1,1],[3,2]])
I = Matrix([[1,0],[0,1]])
print(A)
print(A * 3)
#print(A * I)
print(A * A)
print(A**2)


[ 1 1 
  3 2 ]
[ 3 3 
  9 6 ]
[ 4 3 
  9 7 ]
[ 4 3 
  9 7 ]


New Above

In [ ]:
matrix_A = [[6,4,1],
            [8,5,3],
            [12,4,7]]

matrix_B = [[1,1,1],
            [3,3,3],
            [2,1,3]]

matrix_I = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1]]

In [ ]:
## Determinant Calculator helper function
# ----- Accepts a matrix and a column index, returns that matrix with row 1 and that column removed

def reduce_matrix(matrix: list, col: int):
    
    if col >= len(matrix[0]) or col < 0:
        print("Invalid index passed in argument")
        return
    
    ## Because det() traverses the top row, we will construct the reduced matrix 
    # by defining a new matrix without the top row and the passed column
    ## Each sublist of the 2D list is a row
    
    return_matrix = []
    row_matrix = []
    for i in range(1,len(matrix)):
        row_matrix = []
        
        for j in range(len(matrix[i])):
            if j == col:
                continue
            else:
                row_matrix.extend([matrix[i][j]])
                
            
        return_matrix.append(row_matrix)
    return return_matrix
      

print(reduce_matrix(matrix_I, 0))

In [ ]:
## Determinant Calculator of n x n Matrix:

def det(matrix) -> int:
    
    if len(matrix) != len(matrix[0]):
        print("Matrix is not square")
        return

    if len(matrix) == 1:
        return matrix[0][0] # error here
            
    ## We will traverse across the top row
    
    temp = 0
    for j in range(len(matrix)):
        reduced = reduce_matrix(matrix,j)
        temp += ((-1)**j)*(matrix[0][j] * det(reduced)) # error because of det(reduced)   
    return temp
        

print(det(matrix_I))


In [ ]:
# What would be sufficient for a "linear algebra package"?

# Determinant Calculator
# Eigenvector and Eigenvalue Calculator
# Invertibility Checker
# Inverse Calculator
# Invertibility Checker
# Orthogonality Checker
# Orthonormal Checler
# Diagonalizability Checker
# Reducer to rref
# Dimension calculator
# Linear Dependence Checker of set of vectors
# Inner Product Definition
# Reducer to Basis from set of vectors (Gram-Schmidt Algorithm)
# Normalizer of vectors
# Linear System solver
# Matrix addition and multiplication

# Could I define a theorem checker? Given already defined theorems, check if the passed logic combination is also a theorem. If so, add to knowledge base. 

# Check if a vector (function) is an a vector space


# For eigenvalue calculator, probably gonna have to say:
# 1) Find characteristic polynomial of matrix
#    Gonna have to make a new determinant function in polynomial class form. 
#    For evert diagonal element a, make a polynomial object that is a-x
#    multiply that polynomial by inner matrix as a polynomial
# 2) Find roots of characteristic polynomial
# To easily do this requires factored form of polynomial
# Do this iteratively